In [1]:
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utilities_s7 import *
from utilities_s8 import *

## 7 Incorporating Conditional Information

### 7.1 Approach I

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M Y\right] = 0, \\
&{\mathbb E}\left[M\right] = 1.
\end{align*}

#### Dual problem:

For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}_j}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X) + \sum_{j}\hat{\lambda}_j \cdot YB^j \right] \right)\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

#### Formula for $M$:

\begin{equation*}
M^* = \frac{\exp\left[-\frac{1}{\xi^*}\left(g(X) + \hat{\lambda}^*\cdot f(X)\right)\right]}{\mathbb E \left( \exp\left[-\frac{1}{\xi^*}\left(g(X) + \hat{\lambda}^*\cdot f(X)\right)\right]\right)}
\end{equation*}


In [2]:
# Set model paramters
tol = 1e-12
max_iter = 1000

solver_s7 = StaDivConstraint(tol,max_iter)
approach = 1

# Calculate lower bound for k
print("---Approach I---")
min_k_approach_1 = solver_s7.cal_min_k(approach)['result']
print("Lower Bound for k: %s" % min_k_approach_1)

# Set k
k = min_k_approach_1 * 1.1
result_approach_1 = solver_s7.solve(k,approach)
moment_bound_approach_1 = result_approach_1['result']
print("\n")
print("---Setting and Results---")
print("k = %s" % k)
print("E[Mg(X)]= %s" % moment_bound_approach_1)

---Approach I---
Lower Bound for k: 0.05390640876535269


---Setting and Results---
k = 0.05929704964188796
E[Mg(X)]= 0.0065056198610468005


In [3]:
# Calculate M
temp = np.exp(-1./result_approach_1['ξ'] * (solver_s7.g + solver_s7.f@result_approach_1['λ']))
M = temp/np.mean(temp)

# Calculate transition matrix and stationary distribution under the original empirical probability
P = np.zeros((solver_s7.n_states,solver_s7.n_states))
for i in np.arange(1,solver_s7.n_states+1,1):
    for j in np.arange(1,solver_s7.n_states+1,1):
        P[i-1,j-1] = np.mean(solver_s7.pd_indicator[solver_s7.pd_lag_indicator[:,i-1]][:,j-1]) 
A = P.T - np.eye(solver_s7.n_states)
A[-1] = np.ones(solver_s7.n_states)
B = np.zeros(solver_s7.n_states)
B[-1] = 1.
π = np.linalg.solve(A, B)  

# Calculate transition matrix and staionary distribution under distorted probability
π_tilde = np.zeros_like(π)
for i in np.arange(1,solver_s7.n_states+1,1):
    π_tilde[i-1] = np.mean(M * solver_s7.pd_lag_indicator[:,i-1])
#     π_tilde[i-1] = np.mean(M * solver_s7.pd_indicator[:,i-1])

# Calculate conditional/unconditional moment bound
moment_bound_cond = []
for i in np.arange(1,solver_s7.n_states+1,1):
    temp = np.mean(M[solver_s7.pd_lag_indicator[:,i-1]]*solver_s7.g[solver_s7.pd_lag_indicator[:,i-1]])
    moment_bound_cond.append(temp)
moment_bound_cond = np.array(moment_bound_cond)
moment_bound = np.mean(M*solver_s7.g)

# Calculate conditional/unconditional relative entropy
RE_cond = []
for i in np.arange(1,solver_s7.n_states+1,1):
    temp = np.mean(M[solver_s7.pd_lag_indicator[:,i-1]]*np.log(M[solver_s7.pd_lag_indicator[:,i-1]]))
    RE_cond.append(temp)
RE_cond = np.array(RE_cond)
RE = np.mean(M*np.log(M))

In [4]:
# Print stationary distribution under the original empirical probability
print("--- Distribution (Original) ---")
print(π)

# Print stationary distribution under distorted probability
print("\n")
print("--- Distribution (Distorted) ---")
print(π_tilde)

# Print conditional moment bound
print("\n")
print("--- Conditional Moment Bound ---")
print("E[Mg(X)|state 1] = %s " % moment_bound_cond[0])
print("E[Mg(X)|state 2] = %s " % moment_bound_cond[1])
print("E[Mg(X)|state 3] = %s " % moment_bound_cond[2])

# Print unconditional moment bound
print("\n")
print("--- Unconditional Moment Bound ---")
print("E[Mg(X)] = %s" % moment_bound)

# Print conditional relative entropy
print("\n")
print("--- Conditional Relative Entropy ---")
print("E[MlogM|state 1] = %s " % RE_cond[0])
print("E[MlogM|state 2] = %s " % RE_cond[1])
print("E[MlogM|state 3] = %s " % RE_cond[2])

# Print unconditional moment bound
print("\n")
print("--- Unconditional Relative Entropy ---")
print("E[MlogM] = %s" % RE)

--- Distribution (Original) ---
[0.41654572 0.31240929 0.27104499]


--- Distribution (Distorted) ---
[0.3551099  0.34137148 0.30351862]


--- Conditional Moment Bound ---
E[Mg(X)|state 1] = 0.004312583960382065 
E[Mg(X)|state 2] = 0.006347712922077528 
E[Mg(X)|state 3] = 0.008828316429743868 


--- Unconditional Moment Bound ---
E[Mg(X)] = 0.006505646186358005


--- Conditional Relative Entropy ---
E[MlogM|state 1] = 0.08508894155696599 
E[MlogM|state 2] = 0.06377251191365019 
E[MlogM|state 3] = 0.02939399541834139 


--- Unconditional Relative Entropy ---
E[MlogM] = 0.05929692633325046


### 7.3 Approach III

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\mid\mathfrak{F}_0\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\mid\mathfrak{F}_0\right] \leq \kappa, \\
&{\mathbb E}\left[M  Y\mid\mathfrak{F}_0\right] = 0, \\
&{\mathbb E}\left[M\mid\mathfrak{F}_0\right] = 1.
\end{align*}

#### Dual problem:
For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X) + \hat{\lambda} \cdot Y \right] \right)\mid\mathfrak{F}_0\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

In [5]:
# Set model paramters
tol = 1e-12
max_iter = 1000

solver_s7 = StaDivConstraint(tol,max_iter)
approach = 3
state = 1

# Calculate lower bound for k
print("---Approach III, condition on state %s---" % state)
min_k_approach_3 = solver_s7.cal_min_k(approach,state)['result']
print("Lower Bound for k: %s" % min_k_approach_1)

# Set k
k = min_k_approach_3 * 1.1
moment_bound_approach_3 = solver_s7.solve(k,approach,state)['result']
print("\n")
print("---Setting and Results---")
print("k = %s" % k)
print("E[Mg(X)|state %s]= %s" % (state,moment_bound_approach_3))

---Approach III, condition on state 1---
Lower Bound for k: 0.05390640876535269


---Setting and Results---
k = 0.004387594749274993
E[Mg(X)|state 1]= 0.005635026995385118


## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [15]:
# Set ξ
# ξ = 0.2  # 10% higher than the minimum RE
# ξ = 0.14 # 20% higher than the minimum RE
ξ = 10.

# Set model paramters
tol = 1e-9
max_iter = 1000

solver_s8 = InterDivConstraint(tol,max_iter)

time_start = time.time() 
result = solver_s8.iterate(ξ)
result_upper = solver_s8.iterate(ξ,lower=False)

# Print iteration information
print("--- Iteration Ends ---")
print("ξ = %s" % ξ)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))
# print("Numer of iterations: %s ---" % result['count'])

# # Print converged parameter results
# print("\n")
# print("--- Converged vlues for the lower bound problem ---")
# print("ϵ: %s" % result['ϵ'])
# print("e: %s" % result['e'])
# print("λ: %s" % result['λ'])
# print("μ: %s" % result['μ'])

# # Print E[M|state k]
# print("\n")
# print("--- Check 1 ---")
# print("E[M|state 1] = %s " % result['E_M_cond'][0])
# print("E[M|state 2] = %s " % result['E_M_cond'][0])
# print("E[M|state 3] = %s " % result['E_M_cond'][0])

# # Print two ways of calculating moment bound
# print("\n")
# print("--- Check 2 ---")
# print("E[Mg(X)] = %s (directly using M)" % result['moment_bound'])
# print("E[Mg(X)] = %s (indirectly using μ and RE)" % result['moment_bound_check'])
# print("Difference: %s" % (result['moment_bound']-result['moment_bound_check']))

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(result['P'])

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(result['P_tilde'])

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(result_upper['P_tilde'])


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(result['π'])

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(result['π_tilde'])

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(result['π_tilde'])

# Print relative entropy
print("\n")
print("--- Relative Entropy (lower bound problem) ---")
print("E[MlogM|state 1] = %s " % result['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result['RE_cond'][2])
print("E[MlogM]         = %s " % result['RE'])

# Print relative entropy
print(" ")
print("--- Relative Entropy (Upper bound problem) ---")
print("E[MlogM|state 1] = %s " % result_upper['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_upper['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_upper['RE_cond'][2])
print("E[MlogM]         = %s " % result_upper['RE'])

# Print conditional moment & bounds
print("\n")
print("--- Conditional Moment (Original) ---")
print("E[g(X)|state 1]  = %s " % result['moment_cond'][0])
print("E[g(X)|state 2]  = %s " % result['moment_cond'][1])
print("E[g(X)|state 3]  = %s " % result['moment_cond'][2])
print(" ")
print("--- Conditional Moment (Lower bound) ---")
print("E[Mg(X)|state 1] = %s " % result['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result['moment_bound_cond'][2])
print(" ")
print("--- Conditional Moment (Upper bound) ---")
print("E[Mg(X)|state 1] = %s " % result_upper['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result_upper['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result_upper['moment_bound_cond'][2])

# Print unconditional moment & bounds
print("\n")
print("--- Unconditional Moment (Original) ---")
print("E[g(X)]  = %s " % result['moment'])
print(" ")
print("--- Unconditional Moment (Lower bound) ---")
print("E[Mg(X)] = %s " % result['moment_bound'])
print(" ")
print("--- Unconditional Moment (Upper bound) ---")
print("E[Mg(X)] = %s " % result_upper['moment_bound'])

--- Iteration Ends ---
ξ = 10.0
Time spent: 0.592 seconds ---


--- Transition Probability Matrix (Original) ---
[[0.96341463 0.03658537 0.        ]
 [0.04878049 0.87804878 0.07317073]
 [0.         0.08433735 0.91566265]]
 
--- Transition Probability Matrix (Distorted, lower bound problem) ---
[[0.97856179 0.02143821 0.        ]
 [0.08214167 0.88190482 0.03595351]
 [0.         0.17247155 0.82752845]]
 
--- Transition Probability Matrix (Distorted, upper bound problem) ---
[[0.97833639 0.02166361 0.        ]
 [0.08173756 0.88208616 0.03617627]
 [0.         0.17188513 0.82811486]]


--- Stationary Distribution (Original) ---
[0.41654572 0.31240929 0.27104499]
 
--- Stationary Distribution (Distorted, lower bound problem) ---
[0.76022677 0.19841213 0.0413611 ]
 
--- Stationary Distribution (Distorted, upper bound problem) ---
[0.76022677 0.19841213 0.0413611 ]


--- Relative Entropy (lower bound problem) ---
E[MlogM|state 1] = 0.011489543594734638 
E[MlogM|state 2] = 0.0647095856244338 
E

In [7]:
# Solve the minimization problems over a grid of ξ
tol = 1e-9
max_iter = 1000

solver_s8 = InterDivConstraint(tol,max_iter)

# Grid for ξ
ξ_grid = np.arange(.01,1.01,.01)
μs = np.zeros_like(ξ_grid)
REs = np.zeros_like(ξ_grid)
bounds = np.zeros_like(ξ_grid)
bounds_cond_1 = np.zeros_like(ξ_grid)
bounds_cond_2 = np.zeros_like(ξ_grid)
bounds_cond_3 = np.zeros_like(ξ_grid)
ϵs = np.zeros_like(ξ_grid)

time_start = time.time() 
for i in range(len(ξ_grid)):
    ξ = ξ_grid[i]
    result = solver_s8.iterate(ξ)
    μs[i] = result['μ']
    REs[i] = result['RE']
    bounds[i] = result['moment_bound']
    bounds_cond_1[i] = result['moment_bound_cond'][0]
    bounds_cond_2[i] = result['moment_bound_cond'][1]
    bounds_cond_3[i] = result['moment_bound_cond'][2]
    ϵs[i] = result['ϵ']

print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

Time spent: 25.6455 seconds ---


In [14]:
# Plots for μ and RE
fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Scatter(x=ξ_grid, y=REs, name='RE'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=ξ_grid, y=bounds, name='E[Mg(X)]'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=ξ_grid, y=bounds_cond_1, name='E[Mg(X)|state 1]', line=dict(dash='dash')),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=ξ_grid, y=bounds_cond_2, name='E[Mg(X)|state 2]', line=dict(dash='dash')),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=ξ_grid, y=bounds_cond_3, name='E[Mg(X)|state 3]', line=dict(dash='dash')),
    row=1, col=2
)

# Add lines
fig.update_layout(
    shapes=[dict(type="line", xref="x1", yref="y1",
            x0=0, y0=REs[-1]*1.1, x1=1, y1=REs[-1]*1.1
, line_width=1.5)])

fig.update_layout(height=400, width=1000)
fig.update_xaxes(rangemode="tozero",title_text='ξ')
fig.update_yaxes(rangemode="tozero")

# fig['layout']['xaxis'+str(int(1))].update(range = (0,0.2))
fig['layout']['yaxis'+str(int(1))].update(range = (0.,0.06))
fig['layout']['xaxis'+str(int(2))].update(range = (0.,1.))
fig['layout']['yaxis'+str(int(2))].update(range = (-0.01,0.02))

fig.show()

In [9]:
REs[-1]*1.1

0.0314176714489537